# Cycle GAN

## Build Model

In [1]:
import sys
sys.path.append('../')

from cycle_gan import CycleGAN
from data_loader.reside_data_loader import RESIDEOTSDataLoader as DataLoader
from skimage.measure import compare_psnr, compare_ssim

model = CycleGAN()
dl = DataLoader()

## Train Model

In [ ]:
datasetX, datasetY = dl.train_data()
model.train(5, datasetX, datasetY, 8, 5)

$X \to Y$

In [ ]:
model.generator_g.predict()

$Y \to X$

In [ ]:
model.generator_f.predict()

## Evaluate

In [ ]:
def evaluate(model, datasetX, datasetY):

    psnr_list, ssim_list = [], []
    
    for image_x, image_y in tf.data.Dataset.zip((datasetX, datasetY)).batch(batch_size):
        fake_y = model.predict(image_x)
        for img_y, fak_y in zip(image_y, fake_y):
            psnr_list.append(compare_psnr(img_y, fak_y, data_range=1))
            psnr_list.append(compare_ssim(img_y, fak_y, 3))

    return psnr_list, ssim_list

datasetX, datasetY = dl.test_data()
evaluate(model.generator_g, datasetX, datasetY)
